In [1]:
import DTC.json_read_write as json_read_write
point_cloud = json_read_write.read_point_cloud_from_json("AllcityPC.json")
safe_areas = json_read_write.read_safe_areas_from_json("AllcitySA01.json")
initialization_point = (116.20287663548845, 39.75112986803514)

***We use random.sample to get a random set of trajectories every time this runs***
- furthermore, we take 5% of the points and shift them between 1-200m randomly with a random bearing

In [11]:
import random
from DTC.noise_correction import NoiseCorrection
randomly_elected_trajectories = random.sample(point_cloud.trajectories, 1467)
noise_corrector = NoiseCorrection(safe_areas, initialization_point)

In [ ]:
points_cleaned = 0
total_points = 0
for trajectory in randomly_elected_trajectories:
    labels_of_cleaned_points = noise_corrector.noise_detection(trajectory)
    points_cleaned += len(labels_of_cleaned_points)
    total_points += len(trajectory.points)
print("Points in randomly elected trajectories: ", total_points)
print("Points cleaned: ", points_cleaned)

In [ ]:
# use 5% of 52489 - or 5 % of all the points in the collective pointcloud for all trajectories
from math import ceil
amount_to_shift = ceil(total_points * 0.05)
print("Amount of points to shift: ", amount_to_shift)

list_of_points_in_randomly_elected_trajectories = []
for trajectory in randomly_elected_trajectories:
    list_of_points_in_randomly_elected_trajectories.extend(trajectory.points)

points_to_shift = random.sample(list_of_points_in_randomly_elected_trajectories, amount_to_shift)
print("Amount of points actually shifted: ", len(points_to_shift))

In [18]:
from DTC.distance_calculator import DistanceCalculator
for point in points_to_shift:
    point.set_coordinates(DistanceCalculator.shift_point_with_bearing(point, random.randint(1,200), random.randint(0,359)))
    point.noise = True

- Clean trajectories following shifting. 
After Cleaning, make comparison with the old trajectory points

In [ ]:
labels_of_cleaned_points = []
for trajectory in randomly_elected_trajectories:
    labels_of_locally_cleaned_points = noise_corrector.noise_detection(trajectory)
    labels_of_cleaned_points.extend(labels_of_locally_cleaned_points)

amount_of_cleaned_points = 0
amount_of_noisy_points_cleaned = 0
for label in labels_of_cleaned_points:
    amount_of_cleaned_points += 1
    if label == True:
        amount_of_noisy_points_cleaned += 1

print("Amount of cleaned points: ", amount_of_cleaned_points)
print("Amount of noisy points cleaned: ", amount_of_noisy_points_cleaned)
print("Amount of not noisy points cleaned: ", amount_of_cleaned_points - amount_of_noisy_points_cleaned)
print("Amount of noisy points not cleaned: ", amount_to_shift - amount_of_noisy_points_cleaned)